In [6]:
import torch
import torch.optim as optim
import tqdm.notebook as tqdm

from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.optim.lr_scheduler import StepLR


import os
import sys
sys.path.append('/Users/Matt/projects/gmmvae/')
sys.path.append('/Users/Matt/projects/sgpvae/')

import gmmvae
import entrovae

## Data preparation.

In [7]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST('../data', train=True, download=True,
                               transform=transform)
test_dataset = datasets.MNIST('../data', train=False,
                              transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

## Classifier training.

In [8]:
def train_cls(model, train_loader, optimiser, epoch):
    model.train()
    batch_iter = tqdm.tqdm(enumerate(train_loader), desc='Batch')
    for batch_idx, (x, y) in batch_iter:
        optimiser.zero_grad()
        loss, _ = model.nll(x, y)
        loss.backward()
        optimiser.step()
        
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test_cls(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            loss, output = model.nll(x, y)
            test_loss += loss.item()
            
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(y.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
cls = gmmvae.classifiers.MNISTClassificationNet()
cls_optimiser = optim.Adadelta(cls.parameters(), lr=1.0)
cls_scheduler = StepLR(cls_optimiser, step_size=1, gamma=0.7)

In [10]:
for epoch in range(1, 14+1):
    train_cls(cls, train_loader, cls_optimiser, epoch)
    test_cls(cls, test_loader)
    cls_scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299464
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.909088
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.996428
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.864438
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.966361
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.493584
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.380833
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.339931
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.345949
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.390262
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.227918
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.222801
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.246586
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.114312
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.148051
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.190353
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.249175
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.174220
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.159441
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.118655
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.126431
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.068296
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.114592
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.072888
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.028549
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.010017
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.036521
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.066841
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.044934
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.036041
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.009039
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.024086
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.129806
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.116213
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.093969



KeyboardInterrupt: 

## Set GMMVAE datasets. 

In [11]:
pred_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=False)
cls_output = torch.zeros(len(train_dataset), 10)
with torch.no_grad():
    for batch_idx, (x, y) in enumerate(pred_loader):
        cls_output[batch_idx*64:(batch_idx+1)*64] = cls(x).detach().exp()

In [12]:
class GMMVAEDataset(torch.utils.data.Dataset):
    def __init__(self, x, cls_output):
        self.x = x
        self.cls_output = cls_output
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        return self.x[idx, ...], self.cls_output[idx]
    
gmmvae_dataset = GMMVAEDataset(train_dataset.data, cls_output)
gmmvae_loader = torch.utils.data.DataLoader(gmmvae_dataset, batch_size=64)

In [13]:
from torch import autograd

def train_gmmvae(model, loader, optimiser, epoch):
    model.train()
    train_loss = 0
    batch_iter = tqdm.tqdm(enumerate(loader), desc='Batch')
    
    for batch_idx, (x, pi) in batch_iter:
        optimiser.zero_grad()
        
        x = x.view(-1, 784).float() / 255
        
        loss = -model.elbo(x, pi)
        loss.backward()
        
        for name, param in model.named_parameters():
            if (param.grad != param.grad).any():
                pdb.set_trace()
                print('wtf')
        
        optimiser.step()
        
        train_loss += loss.item()        
        batch_iter.set_postfix(loss=loss.item())
            
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(loader.dataset)))

In [14]:
z_dim = 2
x_dim = 784

encoder = sgpvae.networks.LinearGaussian(x_dim, z_dim, [512, 256], min_sigma=1e-3)
loglikelihood = gmmvae.loglikelihoods.NNBernoulli(z_dim, x_dim, [256, 512])

gmmvae_model = gmmvae.models.GMMVAE(encoder, loglikelihood, z_dim, 10)

## GMMVAE training.

In [15]:
gmmvae_optimiser = optim.Adam(gmmvae_model.parameters())
for epoch in range(1, 14+1):
    train_gmmvae(gmmvae_model, gmmvae_loader, gmmvae_optimiser, epoch)


====> Epoch: 1 Average loss: 2.8252



====> Epoch: 2 Average loss: 2.4922



====> Epoch: 3 Average loss: 2.3915



====> Epoch: 4 Average loss: 2.3333



====> Epoch: 5 Average loss: 2.2953



====> Epoch: 6 Average loss: 2.2684



====> Epoch: 7 Average loss: 2.2488


KeyboardInterrupt: 

In [19]:
if not os.path.exists('./samples'):
    os.makedirs('./samples')

with torch.no_grad():            
    sample = gmmvae_model.sample(
        pi=torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        num_samples=100)
    
    filename = './samples/gmmvae_sample'
    
    if os.path.exists(filename + '.png'):
        i = 1
        while os.path.exists(filename + '_' + str(i) + '.png'):
            i += 1
            
        filename = filename + '_' + str(i) + '.png'
        
    else:
        filename = filename + '.png'
    
    save_image(sample.view(100, 1, 28, 28), filename)